In [0]:
#웹페이지 html소스
import requests
#데이터 추출을 위한 객체 변환
from bs4 import BeautifulSoup as bs
import pandas as pd

In [0]:
# 크롤링 할 사이트
url = 'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

all_data_list = []

def crawling_rent_data(pnum):
    response = requests.get( url + str(pnum))
        
    if response.status_code != 200:
        return False
    
    soup = bs(response.text, 'html.parser')
    tbody = soup.select('table.psboard1 > tbody')
    
    if not tbody:
        return False
    
    data = rent_info(tbody)

    if data:
        # 크롤링해서 가져온 대여소 정보 리스트 저장. 
        all_data_list.extend(data)
        
        #페이지 번호 307까지 존재.
        pnum += 1
        return crawling_rent_data(pnum)
    else:
        return all_data_list

In [0]:
def rent_info(tbody):
    if tbody: 
        trs = tbody[0].find_all('tr')
        rent_data_list = []
        for tr in trs:
            # 대여소 정보를 담아준다.
            info = []
            # 대여소명
            name = tr.select('td.pl10')[0].get_text(strip=True)
            if '.' in name:
                info.append(name.split('.')[0].strip()) #대여소번호
                info.append(name.split('.')[-1].lstrip()) #대여소명
            else:
                # 대여소 정보가 없다면 결측치 처리.
                info.append(pd.np.nan)
                info.append(name)
            # 거치대수
            info.append(tr.select('td.tr')[0].get_text(strip=True))
            # 주소
            info.append(tr.select('td.mhid')[0].get_text(strip=True))
            # 위도, 경도
            geo = tr.find('a')['param-data'].split(',')
            info.append(geo[0])
            info.append(geo[1])
            rent_data_list.append(info)
        return rent_data_list
    else :
        return False

In [0]:
crawling_rent_data(1)

[['1001',
  '광진교 남단 사거리(디지털프라자앞)',
  '15',
  '서울특별시 강동구 구천면로 171 376 남단사거리',
  '37.54180527',
  '127.12471771'],
 ['1002',
  '해공공원(천호동)',
  '10',
  '서울특별시 강동구 올림픽로 702 265',
  '37.54521942',
  '127.12591553'],
 ['1003',
  '해공도서관앞',
  '20',
  '서울특별시 강동구 올림픽로 702 367',
  '37.54391479',
  '127.12545776'],
 ['1004',
  '삼성광나루아파트 버스정류장',
  '10',
  '서울특별시 강동구 상암로3길 77 암사동 534',
  '37.55334854',
  '127.12886047'],
 ['1006',
  '롯데캐슬 115동앞',
  '15',
  '서울특별시 강동구 양재대로 1665 명일동 304',
  '37.55492020',
  '127.14279938'],
 ['1007',
  '암사동 선사유적지',
  '16',
  '서울특별시 강동구 올림픽로 875 암사동 139-2',
  '37.55951691',
  '127.13084412'],
 ['1008',
  '암사역 3번출구(국민은행앞)',
  '10',
  '서울특별시 강동구 올림픽로 지하 776 암사동 555',
  '37.54954910',
  '127.12715149'],
 ['1009',
  '천호역4번출구(현대백화점)',
  '20',
  '서울특별시 강동구 천호대로 지하 997 성내동 62-1',
  '37.53865814',
  '127.12421417'],
 ['101',
  '(구)합정동 주민센터',
  '5',
  '서울특별시 마포구 동교로8길 58 444-12',
  '37.54956055',
  '126.90575409'],
 ['1010',
  '강동세무서',
  '20',
  '서울특별시 강동구 천호대로 1139 길동 467',
  '

In [0]:
all_data_list[:2]

[['1001',
  '광진교 남단 사거리(디지털프라자앞)',
  '15',
  '서울특별시 강동구 구천면로 171 376 남단사거리',
  '37.54180527',
  '127.12471771'],
 ['1002',
  '해공공원(천호동)',
  '10',
  '서울특별시 강동구 올림픽로 702 265',
  '37.54521942',
  '127.12591553']]

In [0]:
header = ['대여소번호', '대여소명', '거치대수', '주소', '위도', '경도']
df = pd.DataFrame.from_records(all_data_list, columns = header)

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls '/content/gdrive/'

'My Drive'


In [0]:
df.to_csv('/content/gdrive/My Drive/bike_rent_service.csv', index=False)

In [0]:
pd.read_csv('/content/gdrive/My Drive/bike_rent_service.csv').head()

,대여소번호,대여소명,거치대수,주소,위도,경도
0,1001.0,광진교 남단 사거리(디지털프라자앞),15,서울특별시 강동구 구천면로 171 376 남단사거리,37.541805,127.124718
1,1002.0,해공공원(천호동),10,서울특별시 강동구 올림픽로 702 265,37.545219,127.125916
2,1003.0,해공도서관앞,20,서울특별시 강동구 올림픽로 702 367,37.543915,127.125458
3,1004.0,삼성광나루아파트 버스정류장,10,서울특별시 강동구 상암로3길 77 암사동 534,37.553349,127.128860
4,1006.0,롯데캐슬 115동앞,15,서울특별시 강동구 양재대로 1665 명일동 304,37.554920,127.142799
